In [58]:
import os
import sys
import pandas as pd
sys.path.append("../")
from pyspark.sql import SparkSession
from pyspark.sql.functions import split
from pyspark.sql import functions as F

In [2]:
spark = (
    SparkSession.builder.appName("local")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/10 08:23:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
consumer_details = spark.read.parquet("../data/tables/synthetic/consumer_user_details.parquet")
merchants = spark.read.parquet("../data/tables/synthetic/tbl_merchants.parquet")
consumer_fraud = pd.read_csv("../data/tables/synthetic/consumer_fraud_probability.csv")
merchants_fraud = pd.read_csv("../data/tables/synthetic/merchant_fraud_probability.csv")
consumer = pd.read_csv("../data/tables/synthetic/tbl_consumer.csv")
transactions = spark.read.parquet("../data/tables/synthetic/transactions/*")

In [4]:
consumer_details.describe() #transaction made by 24081 user_id

summary,user_id,consumer_id
count,499999,499999
mean,250000.0,750895.2123184246
stddev,144337.422959767,433100.42601411033
min,1,4
max,499999,1499995


In [5]:
consumer_fraud.describe()

,user_id,fraud_probability
count,34864.000000,34864.000000
mean,12057.003930,15.120091
std,6963.195641,9.946085
min,1.000000,8.287144
25%,6059.750000,9.634437
50%,12067.500000,11.735624
75%,18091.250000,16.216158
max,24081.000000,99.247380


In [6]:
consumer[['name', 'address', 'state', 'postcode', 'gender', 'consumer_id']] = consumer['name|address|state|postcode|gender|consumer_id'].str.split('|', expand=True)
consumer = consumer.drop(columns=['name|address|state|postcode|gender|consumer_id'])

In [7]:
consumer['consumer_id'] = consumer['consumer_id'].astype(int)

The list of merchants from merchants_df is less than the number of merchants shown in the transaction

In [19]:
details = pd.merge(consumer_details.toPandas(), consumer, on='consumer_id', how='outer')
details

,user_id,consumer_id,name,address,state,postcode,gender
0,108417,4,Michele Kelley,28656 Sims Plaza Suite 036,NSW,2774,Female
1,371406,7,James Williams,3709 Mary River,TAS,7248,Male
2,167772,9,Timothy Ramos,290 Melissa Point Apt. 123,QLD,4694,Male
3,137864,10,Beth Mendoza,5002 Meredith Views,QLD,4426,Female
4,92127,19,Dennis Ramirez,20761 Matthews Via Apt. 694,QLD,4406,Male
...,...,...,...,...,...,...,...
499994,419774,1499983,Juan Sims,15168 Jones Row,WA,6054,Male
499995,25039,1499984,Jenna Serrano,53746 Kenneth Avenue,NSW,1740,Female
499996,396464,1499985,Alyssa Myers,047 William Plain Suite 431,NSW,2549,Undisclosed
499997,46172,1499986,Emily Turner,8118 Kimberly Estate Apt. 757,QLD,4475,Female


In [27]:
consumer_full = pd.merge(details, consumer_fraud, on='user_id', how='inner')
consumer_full['user_id'].nunique()

20128

In [21]:
consumer_full[consumer_full.isnull().any(axis=1)].count()

user_id              0
consumer_id          0
name                 0
address              0
state                0
postcode             0
gender               0
order_datetime       0
fraud_probability    0
dtype: int64

In [25]:
consumer_full_1 = consumer_full.groupby(['user_id', 'consumer_id', 'name', 'address', 'state', 'postcode', 'gender']) \
                                .agg(order_count=('order_datetime', 'count'), avg_fraud_prob=('fraud_probability', 'mean'))
consumer_full_1

,,,,,,,order_count,avg_fraud_prob
user_id,consumer_id,name,address,state,postcode,gender,,
1,1195503,Yolanda Williams,413 Haney Gardens Apt. 742,WA,6935,Female,1,9.805431
2,179208,Mary Smith,3764 Amber Oval,NSW,2782,Female,2,9.834682
3,1194530,Jill Jones MD,40693 Henry Greens,NT,862,Female,1,8.300636
4,154128,Lindsay Jimenez,00653 Davenport Crossroad,NSW,2780,Female,1,9.633302
5,712975,Rebecca Blanchard,9271 Michael Manors Suite 651,WA,6355,Female,3,15.794925
...,...,...,...,...,...,...,...,...
24075,1481905,Adam Ramos,482 Kristen Street,SA,5094,Male,2,18.463737
24076,435413,Dr. Jason Scott,3749 Sierra Wells Suite 093,VIC,3469,Male,1,15.657614
24078,487249,Mark Fischer,31055 Scott Brooks Apt. 953,WA,6566,Male,2,15.048298


In [127]:
merchants_1 = merchants.withColumn('tags', F.lower(merchants['tags'])) \
                        .withColumn('name', F.lower(merchants['name'])).toPandas()

In [128]:
merchants_1['tags'] = merchants_1['tags'].str.replace('[', '(').str.replace(']', ')')
merchants_1['tags'] = merchants_1['tags'].astype('string')

In [129]:
merchants_1[['type', 'rev_level', 'take_rate']] = merchants_1['tags'].str.split(r'\), \(', expand=True)
merchants_1 = merchants_1.drop(columns=['tags'])

In [130]:
merchants_1['type'] = merchants_1['type'].str.replace('((', '', regex=False)
merchants_1['take_rate'] = merchants_1['take_rate'].str.replace('))', '', regex=False)

,name,merchant_abn,type,rev_level,take_rate
0,felis limited,10023283211,"furniture, home furnishings and equipment shop...",e,take rate: 0.18))
1,arcu ac orci corporation,10142254217,"cable, satellite, and other pay television and...",b,take rate: 4.22))
2,nunc sed company,10165489824,"jewelry, watch, clock, and silverware shops",b,take rate: 4.40))
3,ultricies dignissim lacus foundation,10187291046,"watch, clock, and jewelry repair shops",b,take rate: 3.29))
4,enim condimentum pc,10192359162,"music shops - musical instruments, pianos, and...",a,take rate: 6.33))
...,...,...,...,...,...
4021,elit dictum eu ltd,99938978285,"opticians, optical goods, and eyeglasses",b,take rate: 4.50))
4022,mollis llp,99974311662,"books, periodicals, and newspapers",b,take rate: 3.17))
4023,sociosqu corp.,99976658299,shoe shops,a,take rate: 6.57))
4024,commodo hendrerit llc,99987905597,motor vehicle supplies and new parts,a,take rate: 6.82))


In [37]:
merchants_trans = merchants_1.join(transactions, on='merchant_abn')
merchants_trans

merchant_abn,name,tags,user_id,dollar_value,order_id
79417999332,phasellus at company,"([gift, card, nov...",14935,136.06570809815838,23acbb7b-cf98-458...
46451548968,tempus eu ligula ...,((health and beau...,1,72.61581642788431,76bab304-fa2d-400...
89518629617,vulputate velit e...,([tent and awnin...,14936,3.0783487174439297,a2ae446a-2959-41c...
49167531725,felis institute,[[digital goods: ...,1,51.58228625503599,7080c274-17f7-4cc...
31101120643,commodo hendrerit...,"[[cable, satellit...",14936,25.228114942417797,8e301c0f-06ab-45c...
67978471888,magna malesuada c...,[(artist supply a...,2,691.5028234458998,0380e9ad-b0e8-420...
60956456424,ultricies digniss...,"([gift, card, nov...",14936,102.13952056640888,5ac3da9c-5147-452...
47644196714,auctor non feugia...,[[motor vehicle s...,2,644.5220654863093,4e368e44-86f8-4de...
39649557865,arcu morbi institute,([artist supply a...,14938,209.12780951421405,4d78cd01-4bab-494...
88402174457,duis dignissim llc,"([furniture, home...",3,141.0387993699113,c50c957d-ecfc-430...


In [42]:
merchants_trans_1 = merchants_trans \
                    .groupBy('name', 'tags', 'merchant_abn') \
                    .agg(
                        F.count('order_id').alias('order_count'),
                        F.sum('dollar_value').alias('total_dollar_value'),
                    )
merchants_trans_1

name,tags,merchant_abn,order_count,total_dollar_value
eget lacus llp,"[[cable, satellit...",47663262928,10422,1013371.6659596296
arcu vestibulum c...,"((hobby, toy and ...",17158952809,20258,1269279.4663547624
arcu iaculis corp...,"([shoe shops], [a...",10955677986,754,176773.53578258128
mauris aliquam eu...,"([cable, satellit...",75720304166,578,44868.54430167646
vel pede blandit ...,"((opticians, opti...",52535771754,4177,1184849.041883478
pretium neque inc.,([tent and awning...,77013874702,3448,46572.26047940074
nullam scelerisqu...,"[(opticians, opti...",78663389603,978,87128.57419733977
eget llc,"[(stationery, off...",49514072231,1647,1204279.1570783928
diam sed inc.,[[computer progra...,25607153542,461,45733.82825409439
donec dignissim m...,"((telecom), (c), ...",95938358124,126,235756.5269952945


In [45]:
merchants_trans_1 = merchants_trans_1.withColumn('spent_per_order', F.col('total_dollar_value') / F.col('order_count'))


In [46]:
merchants_trans_1.dropDuplicates(subset=['name', 'tags', 'merchant_abn'])
merchants_trans_1.count() #no duplicate merchants

4026

In [50]:
fraud = pd.merge(merchants_trans_1.toPandas(), merchants_fraud, on='merchant_abn')
fraud

,name,tags,merchant_abn,order_count,total_dollar_value,spent_per_order,order_datetime,fraud_probability
0,dictum phasellus in institute,"[(gift, card, novelty, and souvenir shops), (a...",94493496784,99176,9.115636e+06,91.913728,2021-11-26,30.579032
1,lacus aliquam corporation,"[(antique shops - sales, repairs, and restora...",31334588839,1527,9.630683e+06,6306.930472,2021-10-02,42.755301
2,lacus aliquam corporation,"[(antique shops - sales, repairs, and restora...",31334588839,1527,9.630683e+06,6306.930472,2021-12-26,38.361660
3,lacus aliquam corporation,"[(antique shops - sales, repairs, and restora...",31334588839,1527,9.630683e+06,6306.930472,2021-11-26,36.209713
4,lacus aliquam corporation,"[(antique shops - sales, repairs, and restora...",31334588839,1527,9.630683e+06,6306.930472,2021-11-29,35.386213
...,...,...,...,...,...,...,...,...
90,pharetra quisque company,"([jewelry, watch, clock, and silverware shops]...",90918180829,557,5.514145e+06,9899.721335,2022-01-27,33.080273
91,pharetra quisque company,"([jewelry, watch, clock, and silverware shops]...",90918180829,557,5.514145e+06,9899.721335,2022-02-19,30.975819
92,nullam enim sed incorporated,"((tent and awning shops), (e), (take rate: 0.27))",78080443264,1099,2.241564e+04,20.396394,2021-11-26,69.095317
93,ornare limited,"([motor vehicle supplies and new parts], [a], ...",96680767841,31134,9.806731e+06,314.984620,2021-11-26,29.555245


In [51]:
fraud_1 = fraud.groupby(['name', 'tags', 'merchant_abn', 'order_count',	'total_dollar_value', 'spent_per_order']) \
                .agg(fraud_count=('order_datetime', 'count'), avg_fraud_prob=('fraud_probability', 'mean'))

fraud_1

,,,,,,fraud_count,avg_fraud_prob
name,tags,merchant_abn,order_count,total_dollar_value,spent_per_order,,
accumsan corporation,"[(jewelry, watch, clock, and silverware shops), (b), (take rate: 4.65)]",86889657711,78,1.160758e+06,14881.517466,1,63.212226
amet risus inc.,"([furniture, home furnishings and equipment shops, and manufacturers, except appliances], [a], [take rate: 6.82])",79827781481,4798,9.734168e+06,2028.797098,2,29.735159
arcu morbi institute,"([artist supply and craft shops], [c], [take rate: 1.47])",39649557865,21919,9.857402e+06,449.719528,1,28.690204
arcu sed eu incorporated,"[(computer programming , data processing, and integrated systems design services), (b), (take rate: 4.80)]",35909341340,37985,9.528214e+06,250.841492,1,29.133053
arcu sed pc,"((antique shops - sales, repairs, and restoration services), (b), (take rate: 4.19))",11590404675,51,8.108893e+05,15899.790936,1,29.607818
arcu vivamus sit limited,"([shoe shops], [a], [take rate: 5.91])",30275989942,235,3.863379e+04,164.399103,1,63.108593
at foundation,"((antique shops - sales, repairs, and restoration services), (b), (take rate: 3.40))",91880575299,33,6.682598e+05,20250.295557,1,32.994978
commodo ipsum industries,"([jewelry, watch, clock, and silverware shops], [b], [take rate: 4.93])",19492220327,824,8.165223e+06,9909.250618,8,31.958307
diam eu dolor llc,"[(tent and awning shops), (b), (take rate: 4.10)]",90568944804,10732,9.618325e+06,896.228525,3,30.722985


In [131]:
merchants_fraud

,merchant_abn,order_datetime,fraud_probability
0,19492220327,2021-11-28,44.403659
1,31334588839,2021-10-02,42.755301
2,19492220327,2021-12-22,38.867790
3,82999039227,2021-12-19,94.134700
4,90918180829,2021-09-02,43.325517
...,...,...,...
109,80089686333,2021-09-01,67.505811
110,83220249221,2021-11-27,63.512459
111,85482742429,2021-11-27,70.881311
112,11470993597,2021-09-28,63.377344
